## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Desktop/seperate-axis"

In [4]:
y = np.load(dir_path + "/y-10sec.npy", allow_pickle= True)

In [5]:
print(y.shape)


(2392, 2560)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(np.array(y[7]))

array([[0.06787109, 0.10302734, 0.12890625, 0.12207031, 0.08886719,
        0.05371094]])

In [8]:
sum(lbs.zero_crossings(np.array(y[7])))

322

In [9]:
y.shape

(2392, 2560)

In [10]:
def zero_crossing(array):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 7))
    
    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[i]))
        zero_crossing_rate[i][6] = sum(lbs.zero_crossings(np.array(array[i])))
    return zero_crossing_rate

        

In [11]:
zcr_y = zero_crossing(y)

In [12]:
print(zcr_y.shape)

(2392, 7)


In [13]:
zcr_y[0]

array([3.22265625e-02, 4.68750000e-02, 7.91015625e-02, 9.32617188e-02,
       7.71484375e-02, 6.25000000e-02, 2.25000000e+02])

In [14]:
columns_zcr = []
for i in range(zcr_y.shape[1]-1):
    columns_zcr.append("zcr_y{}".format(i))

    
columns_zcr.append("#ofcrossing_y")

print(columns_zcr)

['zcr_y0', 'zcr_y1', 'zcr_y2', 'zcr_y3', 'zcr_y4', 'zcr_y5', '#ofcrossing_y']


In [15]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [16]:
df = dataframe_creator(zcr_y, columns = columns_zcr)

df.reset_index(drop = True, inplace = True)
df

,zcr_y0,zcr_y1,zcr_y2,zcr_y3,zcr_y4,zcr_y5,#ofcrossing_y
0,0.032227,0.046875,0.079102,0.093262,0.077148,0.062500,225.0
1,0.062500,0.104980,0.136719,0.148438,0.111328,0.068848,357.0
2,0.059570,0.095215,0.127930,0.128418,0.102051,0.066406,332.0
3,0.069336,0.107422,0.139648,0.129395,0.091309,0.053223,330.0
4,0.055664,0.082031,0.113770,0.112793,0.086914,0.060547,293.0
...,...,...,...,...,...,...,...
2387,0.025879,0.055664,0.078613,0.104004,0.092285,0.062012,243.0
2388,0.038086,0.058105,0.068359,0.063477,0.046387,0.026367,174.0
2389,0.034180,0.046387,0.057129,0.056152,0.033203,0.020996,139.0
2390,0.031250,0.047363,0.069336,0.084961,0.065918,0.049805,200.0


In [17]:
df.to_csv(dir_path + '/y-axis-zcrs.csv')

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import librosa.display

In [20]:
lbs.feature.chroma_stft(np.array(y[i]), sr= 256, hop_length = 512)[1].mean()

0.4162090720761887

In [21]:
def chromagram_average(array):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= 256, hop_length = 512)[j].mean()
    return pitch

In [22]:
chro_y = chromagram_average(y)

In [23]:
print(chro_y.shape)

(2392, 12)


In [24]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [25]:
col_chro = []
for i in range(chro_y.shape[1]):
    col_chro.append(lst[i]+ "_y")
    
print(col_chro)
print(len(col_chro))

['C_y', 'C♯_y', 'D_y', 'D♯_y', 'E_y', 'F_y', 'F♯_y', 'G_y', 'G♯_y', 'A_y', 'A♯_y', 'B_y']
12


In [26]:
ch_df = dataframe_creator(chro_y,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C_y,C♯_y,D_y,D♯_y,E_y,F_y,F♯_y,G_y,G♯_y,A_y,A♯_y,B_y
0,0.470136,0.524001,0.501777,0.506442,0.441205,0.513634,0.468049,0.513746,0.74467,0.765522,0.816477,1.0
1,0.477178,0.567611,0.591758,0.655693,0.551345,0.570116,0.517426,0.543358,0.850113,0.876113,0.841574,0.980735
2,0.558262,0.53704,0.692622,0.637343,0.584928,0.602367,0.640859,0.745081,0.886922,0.899219,0.792471,0.968087
3,0.583226,0.520141,0.650713,0.549535,0.512378,0.566221,0.663774,0.607156,0.809439,0.953873,0.985399,0.940034
4,0.714813,0.452806,0.672956,0.556356,0.501294,0.484056,0.650126,0.655824,0.702659,0.965236,0.974525,0.869775
...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.864479,0.516312,0.543935,0.571469,0.685944,0.648768,0.631376,0.671095,0.63374,0.657194,0.730945,0.97516
2388,0.760673,0.514705,0.611201,0.734499,0.782912,0.766452,0.734329,0.74648,0.829182,0.755555,0.769118,1.0
2389,0.742545,0.433701,0.473902,0.634994,0.681686,0.655716,0.753698,0.896718,0.813898,0.826274,0.87206,1.0
2390,0.756106,0.463903,0.476654,0.500676,0.635378,0.653088,0.694087,0.740521,0.65158,0.724622,0.832133,1.0


In [27]:
ch_df.to_csv(dir_path + "/y-axis-chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [28]:
def tempo_average(array): # 384, 11
    means= []
    for i in range(array.shape[0]): #384
        means.append(array[i][0:11].mean())
    return np.array(means) # 384
        

In [29]:
def tempogram(array = 0, hop_lenght = 256):
    hop_length = 256
    sr =256
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sr, 
                                            hop_length=hop_length)

    times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)

    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr= sr,
                                              hop_length=hop_length)
    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sr,
                                   hop_length=hop_length)[0]
    return [tempogram, tempo]

In [30]:
tempogram(y[0])[0].shape

(384, 11)

In [31]:
tempogram(y[0])[0][12]

array([0.46548825, 0.46779719, 0.47011427, 0.47243966, 0.47477355,
       0.47711609, 0.47946746, 0.48182784, 0.48419739, 0.4865763 ,
       0.48896473])

In [32]:
y[0].shape

(2560,)

In [33]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 384))
    for i in range(array.shape[0]): #143
        
        _x = tempogram(array[i])[0] #1,2560

        final[i] = tempo_average(_x) #384
        
        
    return final

In [34]:
temp_y = temp_values(y)


In [35]:
temp_y.shape

(2392, 384)

In [36]:
columns_t = []
for i in range(temp_y.shape[1]):
    columns_t.append("tempogram_y{}".format(i))
print(columns_t)

['tempogram_y0', 'tempogram_y1', 'tempogram_y2', 'tempogram_y3', 'tempogram_y4', 'tempogram_y5', 'tempogram_y6', 'tempogram_y7', 'tempogram_y8', 'tempogram_y9', 'tempogram_y10', 'tempogram_y11', 'tempogram_y12', 'tempogram_y13', 'tempogram_y14', 'tempogram_y15', 'tempogram_y16', 'tempogram_y17', 'tempogram_y18', 'tempogram_y19', 'tempogram_y20', 'tempogram_y21', 'tempogram_y22', 'tempogram_y23', 'tempogram_y24', 'tempogram_y25', 'tempogram_y26', 'tempogram_y27', 'tempogram_y28', 'tempogram_y29', 'tempogram_y30', 'tempogram_y31', 'tempogram_y32', 'tempogram_y33', 'tempogram_y34', 'tempogram_y35', 'tempogram_y36', 'tempogram_y37', 'tempogram_y38', 'tempogram_y39', 'tempogram_y40', 'tempogram_y41', 'tempogram_y42', 'tempogram_y43', 'tempogram_y44', 'tempogram_y45', 'tempogram_y46', 'tempogram_y47', 'tempogram_y48', 'tempogram_y49', 'tempogram_y50', 'tempogram_y51', 'tempogram_y52', 'tempogram_y53', 'tempogram_y54', 'tempogram_y55', 'tempogram_y56', 'tempogram_y57', 'tempogram_y58', 'tempo

In [37]:
len(columns_t)

384

In [38]:
df_t = dataframe_creator(temp_y,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 384)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_y0,tempogram_y1,tempogram_y2,tempogram_y3,tempogram_y4,tempogram_y5,tempogram_y6,tempogram_y7,tempogram_y8,tempogram_y9,...,tempogram_y374,tempogram_y375,tempogram_y376,tempogram_y377,tempogram_y378,tempogram_y379,tempogram_y380,tempogram_y381,tempogram_y382,tempogram_y383
0,1.0,0.686563,0.562895,0.523742,0.517973,0.51713,0.511474,0.505797,0.500101,0.494388,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.755648,0.672907,0.663991,0.65613,0.630782,0.623763,0.61672,0.609656,0.602573,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0
2,1.0,0.674322,0.554642,0.527596,0.510687,0.490695,0.48538,0.480044,0.474689,0.469317,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.766844,0.692956,0.665395,0.634099,0.612967,0.606203,0.599415,0.592606,0.585777,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,1.0,0.763386,0.690793,0.679979,0.6628,0.655485,0.648141,0.640774,0.633386,0.625979,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,1.0,0.680386,0.573555,0.56361,0.555539,0.536687,0.530796,0.524884,0.518952,0.513003,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2388,1.0,0.67043,0.518953,0.457321,0.446353,0.438265,0.433577,0.428869,0.424143,0.4194,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0
2389,1.0,0.76254,0.664309,0.644933,0.651004,0.669839,0.662244,0.654627,0.64699,0.639335,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2390,1.0,0.646498,0.48918,0.425762,0.408869,0.403541,0.399258,0.394957,0.390639,0.386305,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0


In [45]:
df_t.to_csv(dir_path + '/y-axis-tempogram.csv')

In [40]:
df_t.isin([0]).sum(axis=0)

tempogram_y0      1
tempogram_y1      1
tempogram_y2      1
tempogram_y3      1
tempogram_y4      1
                 ..
tempogram_y379    1
tempogram_y380    1
tempogram_y381    1
tempogram_y382    1
tempogram_y383    1
Length: 384, dtype: int64

In [41]:
df_t['tempogram_y382'][10]

1.1278219536666627e-17

In [42]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcr_y0,zcr_y1,zcr_y2,zcr_y3,zcr_y4,zcr_y5,#ofcrossing_y,C_y,C♯_y,D_y,D♯_y,E_y,F_y,F♯_y,G_y,G♯_y,A_y,A♯_y,B_y
0,0.032227,0.046875,0.079102,0.093262,0.077148,0.062500,225.0,0.470136,0.524001,0.501777,0.506442,0.441205,0.513634,0.468049,0.513746,0.74467,0.765522,0.816477,1.0
1,0.062500,0.104980,0.136719,0.148438,0.111328,0.068848,357.0,0.477178,0.567611,0.591758,0.655693,0.551345,0.570116,0.517426,0.543358,0.850113,0.876113,0.841574,0.980735
2,0.059570,0.095215,0.127930,0.128418,0.102051,0.066406,332.0,0.558262,0.53704,0.692622,0.637343,0.584928,0.602367,0.640859,0.745081,0.886922,0.899219,0.792471,0.968087
3,0.069336,0.107422,0.139648,0.129395,0.091309,0.053223,330.0,0.583226,0.520141,0.650713,0.549535,0.512378,0.566221,0.663774,0.607156,0.809439,0.953873,0.985399,0.940034
4,0.055664,0.082031,0.113770,0.112793,0.086914,0.060547,293.0,0.714813,0.452806,0.672956,0.556356,0.501294,0.484056,0.650126,0.655824,0.702659,0.965236,0.974525,0.869775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.025879,0.055664,0.078613,0.104004,0.092285,0.062012,243.0,0.864479,0.516312,0.543935,0.571469,0.685944,0.648768,0.631376,0.671095,0.63374,0.657194,0.730945,0.97516
2388,0.038086,0.058105,0.068359,0.063477,0.046387,0.026367,174.0,0.760673,0.514705,0.611201,0.734499,0.782912,0.766452,0.734329,0.74648,0.829182,0.755555,0.769118,1.0
2389,0.034180,0.046387,0.057129,0.056152,0.033203,0.020996,139.0,0.742545,0.433701,0.473902,0.634994,0.681686,0.655716,0.753698,0.896718,0.813898,0.826274,0.87206,1.0
2390,0.031250,0.047363,0.069336,0.084961,0.065918,0.049805,200.0,0.756106,0.463903,0.476654,0.500676,0.635378,0.653088,0.694087,0.740521,0.65158,0.724622,0.832133,1.0


In [43]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcr_y0,zcr_y1,zcr_y2,zcr_y3,zcr_y4,zcr_y5,#ofcrossing_y,C_y,C♯_y,D_y,...,tempogram_y374,tempogram_y375,tempogram_y376,tempogram_y377,tempogram_y378,tempogram_y379,tempogram_y380,tempogram_y381,tempogram_y382,tempogram_y383
0,0.032227,0.046875,0.079102,0.093262,0.077148,0.062500,225.0,0.470136,0.524001,0.501777,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.062500,0.104980,0.136719,0.148438,0.111328,0.068848,357.0,0.477178,0.567611,0.591758,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0
2,0.059570,0.095215,0.127930,0.128418,0.102051,0.066406,332.0,0.558262,0.53704,0.692622,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.069336,0.107422,0.139648,0.129395,0.091309,0.053223,330.0,0.583226,0.520141,0.650713,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.055664,0.082031,0.113770,0.112793,0.086914,0.060547,293.0,0.714813,0.452806,0.672956,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.025879,0.055664,0.078613,0.104004,0.092285,0.062012,243.0,0.864479,0.516312,0.543935,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2388,0.038086,0.058105,0.068359,0.063477,0.046387,0.026367,174.0,0.760673,0.514705,0.611201,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0
2389,0.034180,0.046387,0.057129,0.056152,0.033203,0.020996,139.0,0.742545,0.433701,0.473902,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2390,0.031250,0.047363,0.069336,0.084961,0.065918,0.049805,200.0,0.756106,0.463903,0.476654,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0


In [44]:
final_df.to_csv(dir_path + '/y-axis-final-signal_processing.csv')